In [1]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
"LLMs-from-scratch/main/ch02/01_main-chapter-code/"
"the-verdict.txt")

file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x103f27650>)

In [2]:
with  open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character: ", len(raw_text))
print(raw_text[:99])

Total number of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
import re
text = "Hello, world. This, os a test."
result = re.split(r"(\s)", text)
result

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'os', ' ', 'a', ' ', 'test.']

In [4]:
# let's modify the tokenization
result = re.split(r"([,.]|\s)", text)
result

['Hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 ' ',
 'This',
 ',',
 '',
 ' ',
 'os',
 ' ',
 'a',
 ' ',
 'test',
 '.',
 '']

In [5]:
# we want to strip the whitespace
result = [item for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'This', ',', 'os', 'a', 'test', '.']

In [7]:
text = "Hello, world. Is this -- a test?"
result =  re.split(r"([,. :;?_!'()\"]-- |\s)", text)
result = [item for item in result if item.strip()]
result

['Hello,', 'world.', 'Is', 'this', ' -- ', 'a', 'test?']

In [10]:
# let's apply this tokenizer to our raw text
preprocessed = re.split(r"([,. :;?_!'()\"]--|\s)", raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(f"The length of the vocabulary is : {len(preprocessed)}")

The length of the vocabulary is : 3646


In [11]:
# first  30 tokens
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius--though', 'a', 'good', 'fellow', 'enough--so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that,', 'in', 'the', 'height', 'of', 'his', 'glory,']


### create  a token id for each token

In [12]:
# create a vocabulary for the preprocessed text
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(f"The length of the vocabulary is: {vocab_size}")

The length of the vocabulary is: 1486


In [14]:
#  now  create the  vocabulary to have each word mapped to a unique id
vocab = {token: integer for integer, token in enumerate(all_words)}
# print a few output of tokens 
for i, item in enumerate(vocab.items()):
    print(item)
    if i  >= 20:
        break

('!--', 0)
('"--', 1)
('"Ah,', 2)
('"Ah--I', 3)
('"Be', 4)
('"By', 5)
('"Come', 6)
('"Destroyed', 7)
('"Don\'t', 8)
('"Gisburns"', 9)
('"Grindles."', 10)
('"Hang', 11)
('"Has', 12)
('"How', 13)
('"I', 14)
('"I\'d', 15)
('"If', 16)
('"It', 17)
('"It\'s', 18)
('"Jack', 19)
('"Money\'s', 20)


In [20]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        # preprocessed  = re.split(r'([,.?_!"()\']|--\s)', text)
        preprocessed = re.split(r"([,. :;?_!'()\"]--|\s)", text)
        preprocessed = [
            item.strip()  for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = "  ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return  text

In [24]:
new_text  =  """
    It's the last he painted, you know, Mrs. Gisborn said that with pardonable pride.
    """
preprocessed_2 = re.split(r"([,. :;?_!'()\"]--|\s)", new_text)
# sort it and add it to all words
new_vocab =  sorted(set(preprocessed_2))
all_words = all_words + new_vocab
# assign new ids to new words in the vocab
vocab = {token: integer for integer, token in enumerate(all_words)}


tokenizer =  SimpleTokenizerV1(vocab)

# convert the  text to tokens and ids
ids = tokenizer.encode(new_text)
print(ids)

[1490, 1500, 1494, 1492, 1495, 1502, 1493, 1491, 1489, 1498, 1499, 1501, 1496, 1497]


In [25]:
len(all_words)

1503

In [26]:
# Let's try decoding  the ids
print(tokenizer.decode(ids))

It's  the  last  he  painted,  you  know,  Mrs.  Gisborn  said  that  with  pardonable  pride.


we had an issue initially when trying to pass a new text that contains unknown words.

We ran into the issue unknown words. 

Next we're going to deal with this issue by extending our vocabulary

In [27]:
# extend the vocabulary to contain new tokens
all_tokens  = sorted(set(preprocessed))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1488


In [28]:
# check the last five entries in our vocab
for idx, toks in enumerate(list(vocab.items())[-5:]):
    print(toks) 

('younger', 1483)
('your', 1484)
('yourself', 1485)
('<|endoftext|>', 1486)
('<|unk|>', 1487)


In [29]:
# let's update our tokenizer class
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r"([,.:;?_!'()\"]|--|\s)", text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r"\s+([,.:;?!'()\"])", r'\1', text)
        return text

In [30]:
text_1 = "Hello, do you like tea?"
text_2 = "In the sunlit terraces of the palace"
text = " <|endoftext|> ".join((text_1, text_2))
text
 

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace'

In [31]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1487, 1487, 489, 1478, 843, 1277, 1487, 1486, 117, 1292, 1257, 1286, 965, 1292, 1487]


In [32]:
# let's see if we can get the detokenize form of the text in the decoded form
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|> <|unk|> do you like tea <|unk|> <|endoftext|> In the sunlit terraces of the <|unk|>


# Byte pair encoding

In [34]:
from importlib.metadata import version
import tiktoken
print("tiktoken version: ", version("tiktoken"))

tiktoken version:  0.8.0


In [35]:
# now use the new tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

text  = (
    "Hello, do you like tea? <|endoftext|> in the sunlit terraces of someunknownPlace"
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers) 

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 287, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271]


In [36]:
# let's try to decode the text from the integers
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> in the sunlit terraces of someunknownPlace
